In [1]:
import os
from river import datasets
from river import metrics
from river import tree
from river import ensemble
from river import evaluate
from river import compose
from river import naive_bayes
from river import anomaly
from river import compose
from river import datasets
from river import metrics
from river import preprocessing
from confluent_kafka import Producer,Consumer
import certifi
import time
import json
import pandas
import mlflow

In [2]:
user= os.environ['KAFKA_USER_NAME']
password= os.environ['KAFKA_PASSWORD']
bsts= os.environ['KAFKA_BOOTSTRAP_SERVERS']
topic = 'model_updates'
conf = {'bootstrap.servers': bsts,
            'sasl.mechanism': 'PLAIN',
            'security.protocol': 'SASL_SSL',
            'ssl.ca.location': certifi.where(),
            'sasl.username': user,
            'sasl.password': password,            
            'message.max.bytes': 8000000,
            'linger.ms': 100,
            'client.id': 'model-icde-2023'}
producer = Producer(conf) 

KeyError: 'kafka_username'

In [ ]:
import codecs
import pickle
import sys

def get_model_artifact(run_id):
    local_path = client.download_artifacts(run_id, "models/model.pkl", '/tmp/')
    with open(local_path, 'rb') as f:
        m_a = pickle.load(f)
        return m_a

def publish_model_update_to_model_endpoint(run_id,model_version):
    m_a = get_model_artifact(run_id)

    pkd = pickle.dumps(m_a)
    pickled = codecs.encode(pkd, "base64").decode()
    v=model_version
    model_json={'m':pickled,'v':v}
    
    msg = json.dumps(model_json)
    print(sys.getsizeof(msg))
    producer.produce(topic, value=msg, key=str(v))
    producer.flush()


In [ ]:
auc = metrics.ROCAUC()
f1 = metrics.F1()
recall = metrics.MicroRecall()


In [ ]:

def generate_run(exp_id,index,model_artifact):
    global auc
    global f1
    global recall
    idx=0
    run_name = f'run{index}'
    
    with mlflow.start_run(experiment_id=exp_id, run_name=run_name) as run:
        # Retrieve run id
        RUN_ID = run.info.run_id
        size=100 * (index+1)
        max_size=100
        skip_size = size - max_size
        dataset = datasets.MaliciousURL()
        data = dataset.take(size)
        cnt = 0
        if skip_size == 0:
            print(f'First Time Building Model')
            for f, y in data:
                model_artifact = model_artifact.learn_one(f,y)
                cnt = cnt + 1
        else:
            for f, y in data:
                cnt = cnt + 1
                if(cnt==skip_size):
                    break
            print(f'Skipped{skip_size}')
            for f, y in data:
                model_artifact = model_artifact.learn_one(f,y)
                cnt = cnt + 1

            
        # Track parameters
        mlflow.log_param("leaf_prediction", "mc")
        mlflow.log_metric("ROCAUC", auc.get())
        mlflow.log_metric("f1", auc.get())
        mlflow.log_metric("recall", auc.get())
        
        with open('/tmp/model.pkl', 'wb') as handle:
            pickle.dump(model_artifact, handle, protocol=pickle.HIGHEST_PROTOCOL)
        mlflow.log_artifact('/tmp/model.pkl','models')
        return run.info

In [ ]:
def get_latest_model_version_from_model_registry(registered_model):
    versions = registered_model.latest_versions
    current_version = None
    if(len(versions)>1):
        current_version = versions[1]
    else:
        current_version = versions[0]

    return current_version

#model_name = current_version.name
#model_version = current_version.version
#reg_model_run_id = current_version.run_id


In [ ]:
from mlflow import MlflowClient
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
client = MlflowClient()
EXPERIMENT_NAME = 'AdaptiveRandomForestExperiment-03-27-v3'
model_name = EXPERIMENT_NAME
exp = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
model_artifact = None
if not exp:
    EXPERIMENT_ID = mlflow.create_experiment(EXPERIMENT_NAME)
    model_artifact = ensemble.AdaptiveRandomForestClassifier(leaf_prediction="mc")
else:
    EXPERIMENT_ID = exp.experiment_id


In [ ]:
index=0
#First time creating model
model_artifact = ensemble.AdaptiveRandomForestClassifier(leaf_prediction="mc")
run = generate_run(EXPERIMENT_ID,index,model_artifact)

registered_model = None
try:
    registered_model = client.get_registered_model(model_name)
except:
     client.create_registered_model(model_name)
registered_model = client.get_registered_model(model_name)
result = client.create_model_version(
    name=model_name,
    source='/tmp/model.pkl',
    run_id=run.run_id,
) 
registered_model = client.get_registered_model(model_name)
current_model_version = get_latest_model_version_from_model_registry(registered_model)
publish_model_update_to_model_endpoint(run.run_id,current_model_version.version)

In [ ]:
for index in range(1,5):
    print(f'Currently on iteration {index}')
    current_model_version = get_latest_model_version_from_model_registry(registered_model)
    model_artifact = get_model_artifact(current_model_version.run_id)
    run = generate_run(EXPERIMENT_ID,index,model_artifact)
    registered_model = None
    try:
        registered_model = client.get_registered_model(model_name)
    except:
         client.create_registered_model(model_name)

    result = client.create_model_version(
        name=model_name,
        source='/tmp/model.pkl',
        run_id=run.run_id,
    ) 
    registered_model = client.get_registered_model(model_name)
    current_model_version = get_latest_model_version_from_model_registry(registered_model)
    publish_model_update_to_model_endpoint(run.run_id,current_model_version.version)


In [ ]:
import codecs
import pickle
import sys

pkd = pickle.dumps(model_artifact)
pickled = codecs.encode(pkd, "base64").decode()
model_instance = pickle.loads(codecs.decode(pickled.encode(), "base64"))
print(model_instance)


In [ ]:
v=1
model_json={'m':pickled,'v':v}

msg = json.dumps(model_json)
sys.getsizeof(msg)
producer.produce(topic, value=msg, key=str(v))
producer.flush()


In [ ]:
v=2
model_json={'m':pickled,'v':v}
producer.produce(topic, value=json.dumps(model_json), key=str(v))
producer.flush()


In [ ]:
v=3
model_json={'m':pickled,'v':v}
producer.produce(topic, value=json.dumps(model_json), key=str(v))
producer.flush()

In [ ]:
v=4
model_json={'m':pickled,'v':v}
producer.produce(topic, value=json.dumps(model_json), key=str(v))
producer.flush()

## Test the model init and predict function

```
import os
import model
model.init()
#model_test.init_get_latest_model()
x = {"2": 1.0, "4": 0.0788382, "5": 0.131034, "6": 0.117647, "10": 1.0, "11": 0.142857, "16": 0.4, "17": 0.830283, "18": 0.83965, "19": 0.583194, "20": 1.0, "21": 0.285713, "22": 0.00595238, "23": 0.00595238, "36": 1.0, "37": 1.0, "44": 1.0, "45": 1.0, "54": 1.0, "56": 1.0, "62": 1.0, "64": 1.0, "66": 1.0, "68": 1.0, "70": 1.0, "72": 1.0, "74": 1.0, "75": 0.25, "76": 1.0, "77": 0.166667, "79": 0.0769231, "81": 0.05, "82": 1.0, "84": 1.0, "86": 1.0, "88": 1.0, "90": 1.0, "92": 1.0, "94": 1.0, "96": 1.0, "102": 1.0, "104": 1.0, "106": 1.0, "108": 1.0, "110": 1.0, "112": 1.0, "131": 1.0, "133": 1.0, "139": 1.0, "141": 1.0, "143": 1.0, "145": 1.0, "147": 1.0, "149": 1.0, "253": 1.0, "260": 1.0, "277": 1.0, "304": 1.0, "305": 1.0, "425": 1.0, "521": 1.0, "673": 1.0, "674": 1.0, "675": 1.0, "676": 1.0, "731": 1.0, "732": 1.0, "733": 1.0, "1365": 1.0, "1488": 1.0, "2098": 1.0, "6027": 1.0, "6028": 1.0, "6029": 1.0, "6030": 1.0, "6031": 1.0, "9989": 1.0, "18073": 1.0, "18074": 1.0, "18077": 1.0, "18078": 1.0, "18079": 1.0, "18080": 1.0, "155153": 1.0, "155154": 1.0, "155155": 1.0, "155156": 1.0, "155157": 1.0, "155158": 1.0, "155159": 1.0, "155160": 1.0, "155161": 1.0, "155163": 1.0, "155164": 1.0, "155165": 1.0, "155166": 1.0, "155167": 1.0, "155168": 1.0, "155169": 1.0, "155170": 1.0, "155171": 1.0, "155172": 1.0, "155173": 1.0, "155174": 1.0, "155175": 1.0, "155176": 1.0, "155177": 1.0, "155178": 1.0, "155179": 1.0, "155180": 1.0, "155181": 1.0, "155182": 1.0, "155183": 1.0, "155184": 1.0, "155185": 1.0, "155186": 1.0, "155187": 1.0, "155188": 1.0, "155189": 1.0, "155190": 1.0, "155191": 1.0, "155192": 1.0, "155193": 1.0, "155194": 1.0, "155195": 1.0, "155196": 1.0, "155197": 1.0, "155198": 1.0, "155199": 1.0, "155200": 1.0, "155201": 1.0, "155202": 1.0, "155203": 1.0, "155204": 1.0, "155205": 1.0, "155206": 1.0, "155207": 1.0, "155208": 1.0, "155209": 1.0, "155210": 1.0, "155211": 1.0, "155212": 1.0, "155213": 1.0, "500481": 1.0}
model.predict(x)
```